In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import seaborn as sb

vgsales = pd.read_csv("vgsales.csv")
vgsales = pd.DataFrame(vgsales)

vgsales['Year'] = vgsales['Year'].fillna(-1)
vgsales['Year'] = vgsales['Year'].astype(int)
vgsales.set_index('Rank', inplace=True, drop=True)
vgsales

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...
16596,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16598,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,0.00,0.00,0.00,0.00,0.01


# Objetivo do Trabalho

O objetivo desse trabalho será fazer uma análise geral de diversos aspectos do mercado de jogos, que depois irá se aprofundar em determinados pontos específicos, como a comparação de diversas franquias famosas. 

# A dinâmica do mercado

Primeiramente, iremos analisar como a dinâmica do mercado mudou ao longo do tempo. Faremos isso através de uma chamada "bar chart race", um tipo de animação onde cada frame representa um ano específico, com o gráfico de barras representando a quantidade as vendas de cada gênero. Dessa forma, será possível entender os períodos em que determinados gêneros foram os mais populares, e tentaremos descobrir se há algum tipo de tendência ou padrão no mercado de jogos em relação a essa popularidade com o passar do tempo. 

## Manipulação dos dados

Entretanto, para montar uma visualização como essa, é primeiro preciso tratar a base de dados original. Como se pode ver na tabela a seguir, a quantidade de vendas retratada na tabela não representa a quantidade total de vendas cumulativas até aquele ano, e sim as vendas específicas que ocorreram naquele ano (ou seja, os dados de vendas de 2016 não terão nenhuma informação sobre o ano de 2015). Além disso, há ainda diversos jogos cujo ano de lançamento não está disponível na base. Esses jogos serão descartados na análise que será feita posteriormente.

In [2]:
vgsales[(vgsales["Year"] != -1)].groupby(["Year"],as_index=False)[["Global_Sales"]].sum().head()

,Year,Global_Sales
0,1980,11.38
1,1981,35.77
2,1982,28.86
3,1983,16.79
4,1984,50.36


Agora, faremos uma série de operações para criar um novo dataframe, que contém as vendas cumulativas de cada gênero por ano. Para isso, criaremos um subset do dataframe original através do comando .groupby, agrupando os dados pelo ano e gênero de cada jogo, e somando o valor de "Global_Sales" de cada combinação. Além disso, serão definidas algumas variáveis que serão utilizadas posteriormente no processo.

In [3]:
vgsales_year = vgsales[(vgsales["Year"] != -1)].groupby(["Year", "Genre"],as_index=False)[["Global_Sales"]].sum()

genres = set()
years = []
cumulative = []
genreslist = []

vgsales_year.head(10)

,Year,Genre,Global_Sales
0,1980,Action,0.34
1,1980,Fighting,0.77
2,1980,Misc,2.71
3,1980,Shooter,7.07
4,1980,Sports,0.49
5,1981,Action,14.84
6,1981,Platform,6.93
7,1981,Puzzle,2.24
8,1981,Racing,0.48
9,1981,Shooter,10.04


Entretanto, esse método possui um problema. Vejamos a tabela a seguir:

In [4]:
vgsales_year_sorted = vgsales_year.sort_values(["Genre", "Year"])
vgsales_year_sorted[vgsales_year_sorted.Genre.str.contains("Adventure")].head()

,Year,Genre,Global_Sales
20,1983,Adventure,0.40
47,1987,Adventure,4.38
76,1991,Adventure,2.24
88,1992,Adventure,12.24
100,1993,Adventure,0.07


Observe o "pulo" entre o ano de 1983 e 1987. Caso isso não seja tratado, a animação final terá problemas de "descontinuidade", onde uma barra existente em um determinado ano para de existir no ano seguinte. Portanto, teremos que levar isso em conta quando implementarmos a lógica da criação do novo dataset.

Com o subset e as variáveis auxiliares prontas, podemos começar as operações. Primeiramente, será criado um conjunto de todos os gêneros diferentes (a variável "genres"), que será usada em um for loop que fará a "varredura" de cada linha da tabela vgsales_year e atribuirá os valores de venda acumulada corretamente para cada ano e gênero distinto. Por último, criaremos um dataframe com base nesses dados, que será utilizado para a animação.

In [5]:
for tupla in vgsales_year.itertuples():
    genres.add(tupla.Genre) # Criação do conjunto genres


for genre in genres:
    sales = 0 # Reinicia o valor das vendas cumulativas a 0, quando muda de gênero
    for tupla in vgsales_year.itertuples():
        if tupla.Genre == genre: # Checa se está analisando o gênero correto
            if len(years) != 0:
                while (years[-1] + 1 != tupla.Year) and (genreslist[-1] == tupla.Genre): # Loop que corrige os "pulos" de anos
                    years.append(years[-1] + 1)
                    cumulative.append(sales)
                    genreslist.append(tupla.Genre)
                    
            years.append(tupla.Year)
            sales += tupla.Global_Sales # Representa o total de vendas cumulativas do gênero específico até aquele ano
            cumulative.append(sales)
            genreslist.append(tupla.Genre)


dic = {"Year": years,
       "Genre": genreslist,
       "Cumulative_Sales": cumulative
       } # Criação do dicionário

df = pd.DataFrame(dic) # Conversão a Dataframe

df[df.Genre.str.contains("Adventure")].head()

,Year,Genre,Cumulative_Sales
149,1983,Adventure,0.40
150,1984,Adventure,0.40
151,1985,Adventure,0.40
152,1986,Adventure,0.40
153,1987,Adventure,4.78


Como pode ser visto na tabela acima, a lógica aparenta ter dado certo! Agora iremos criar a palheta de cores. 

Algo importante a se notar é que o comportamento padrão do argumento *colors* de um gráfico de barras atribui as cores se baseando na "posição" de cada barra (por exemplo, a primeira barra é sempre azul, a segunda é sempre vermelha). Como nosso objetivo é criar uma animação com base no tempo, faz mais sentido que as cores sejam atribuídas ao *gênero* que a barra representa, para que a comparação entre anos diferentes seja muito mais intuitiva. Portanto, criaremos uma palheta de cores genérica para atribuir a cada gênero específico.


In [6]:
hex = ['#D2D8B3', '#C38D94', '#C3D350', '#A1E8AF',
     '#3C91E6', '#FE5F55', '#90DDF0', '#808F85',
     '#F4B942', '#6A994E', '#747C92', '#E086D3']
colors = dict(zip(genres, hex))
print(colors)

{'Sports': '#D2D8B3', 'Action': '#C38D94', 'Fighting': '#C3D350', 'Misc': '#A1E8AF', 'Adventure': '#3C91E6', 'Strategy': '#FE5F55', 'Role-Playing': '#90DDF0', 'Simulation': '#808F85', 'Shooter': '#F4B942', 'Puzzle': '#6A994E', 'Racing': '#747C92', 'Platform': '#E086D3'}


## Criando o Gráfico

Agora que toda a preparação está pronta, está na hora de criar a função que desenhará o gráfico. O código foi inspirado no tutorial presente no link <https://towardsdatascience.com/bar-chart-race-in-python-with-matplotlib-8e687a5c8a41>, e sua funcionalidade será descrita através de comentários dentro da code chunk. 

In [7]:
fig, ax = plt.subplots(figsize = (15, 9)) # Cria e define o tamanho do plot
plt.close()
def draw_barchart(year):
        dff = (df[df["Year"].eq(year)].sort_values(by="Cumulative_Sales", ascending=True)) # Ordena os dados com base nas vendas
        ax.clear() # Certifica-se que o plot está vazio antes dos dados serem inseridos

        ax.barh(dff["Genre"], dff["Cumulative_Sales"], color = [colors[x] for x in dff["Genre"]]) # Define os eixos e atribui as cores para cada gênero
        dx = dff["Cumulative_Sales"].max() / 200 # Valor usado para posicionar de forma melhor os textos de cada barra

        for i, (sales, genre) in enumerate(zip(dff["Cumulative_Sales"], dff["Genre"])): # Loop que insire o texto específico de cada barra
                ax.text(sales-dx, i-.07, genre, size=13, weight=600, ha="right", va="bottom")
                ax.text(sales-dx, i,  "  " + str(round(sales)), size=13, ha="left", va="center")

        ax.text(1, 0.4, year, transform=ax.transAxes, color='#777777', size=46, ha="right", weight=800) # Mostra o ano do qual os dados estão sendo analisados
        ax.text(0, 1.06, "Vendas de jogos (em milhões)", transform=ax.transAxes, size=12, color="#777777", ha="left", va="center") # Legenda

        # Os comandos a seguir modificam a aparência dos eixos e de seus "ticks"
        ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}')) 
        ax.xaxis.set_ticks_position("top")
        ax.tick_params(axis="x", colors="#333333", labelsize=12)
        ax.set_yticks([])
        ax.margins(0, 0.01)
        ax.grid(which="major", axis="x", linestyle="-")
        ax.set_axisbelow(True)

        ax.text(0, 1.12, "Venda total acumulada de jogos por gênero, de 1980 a 2015", transform=ax.transAxes, size=24, weight=600, ha="left", va="top") # Título
        plt.box(False) # Remover a "borda" do gráfico


Com *draw_barchart* preparado, basta utilizar a função *FuncAnimation*, que irá invocar a função especificada em seus argumentos repetidas vezes, e usar *HTML()* para mostrar a animação que foi feita.

In [8]:
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1980, 2016)) # As frames representam o argumento que será "enviado" para a função especificada
animation = HTML(animator.to_jshtml())
plt.close()

In [9]:
animation # A animação está em uma chunk diferente para evitar que o jupyter também mostre como output um plot vazio, que é gerado durante a aplicação das funções draw_barchart

Com isso, a animação está completa, e nos permite perceber alguns movimentos interessantes na dinâmica do mercado ao longo do tempo.

## Análise das dinâmicas

Primeiramente, iremos analisar o comportamento do gênero de maior popularidade. Os jogos de plataforma se mantiverem na liderança por um período significante de tempo, de 1985 até 2005, ano em que foram ultrapassados pelos jogos de ação. A pergunta que vem a cabeça é: o que causou essa mudança?

O ano de 1985 é um ano extremamente importante na indústria de jogos. É nele em que o primeiro console de mesa da Nintendo, o Nintendo Entertainment System (NES), foi lançado ao público, junto com o jogo de plataforma "Super Mario Bros", um enorme sucesso que estabeleceu diversas normas e regras de design que inspiraram as futuras gerações de jogos do mesmo gênero.

Esse lançamento, unido com suas diversas sequências (dando destaque a Super Mario Bros 3, Super Mario World e Super Mario Land), outras franquias da Nintendo (como Donkey Kong Country), e franquias rivais (como Sonic), consolidaram o gênero de plataforma como o mais popular no universo de consoles 2D.



In [10]:
vgsales[(vgsales.Genre.str.contains("Platform")) & (vgsales.Year < 1996)].head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
19,Super Mario World,SNES,1990,Platform,Nintendo,12.78,3.75,3.54,0.55,20.61
22,Super Mario Land,GB,1989,Platform,Nintendo,10.83,2.71,4.18,0.42,18.14
23,Super Mario Bros. 3,NES,1988,Platform,Nintendo,9.54,3.44,3.84,0.46,17.28
58,Super Mario All-Stars,SNES,1993,Platform,Nintendo,5.99,2.15,2.12,0.29,10.55
72,Donkey Kong Country,SNES,1994,Platform,Nintendo,4.36,1.71,3.00,0.23,9.30
97,Super Mario Bros. 2,NES,1988,Platform,Nintendo,5.39,1.18,0.70,0.19,7.46
144,Sonic the Hedgehog 2,GEN,1992,Platform,Sega,4.47,1.20,0.16,0.19,6.03
185,Super Mario Land 3: Wario Land,GB,1994,Platform,Nintendo,2.49,0.98,1.57,0.15,5.19


Naturalmente, esse "reinado" do gênero de plataforma não durou para sempre, e tudo começou a mudar a partir do ano de 1996, ano onde começou a primeira geração dos "consoles 3D" (PS1, XBOX, N64). 

O ambiente 3D se provou uma plataforma de desenvolvimento que incentivava os desenvolvedores a se inovarem. As inúmeras técnicas acumuladas através dos anos para jogos 2D foram subitamente descartadas, ao mesmo tempo em que infinitas possibilidades foram abertas. Apesar de diversos novos jogos de plataforma terem sido um sucesso, como Super Mario 64 e os jogos da franquia Crash Bandicoot, a maioria das desenvolvedoras decidiu se arriscar nesse mercado volátil, com o objetivo de encontrar a nova "fórmula" para o sucesso, se aventurando em gêneros que não eram tão famosos no mercado, como ação e esportes. 

Podemos ver bem isso nas tabelas abaixo:

In [11]:
vgsales[(vgsales.Genre.str.contains("Platform")) & (vgsales.Year > 1995) & (vgsales.Year < 2006)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
47,Super Mario 64,N64,1996,Platform,Nintendo,6.91,2.85,1.91,0.23,11.89
60,Super Mario 64,DS,2004,Platform,Nintendo,5.08,3.11,1.25,0.98,10.42
96,Crash Bandicoot 2: Cortex Strikes Back,PS,1997,Platform,Sony Computer Entertainment,3.78,2.17,1.31,0.31,7.58
107,Crash Bandicoot 3: Warped,PS,1998,Platform,Sony Computer Entertainment,3.68,1.75,1.42,0.28,7.13
117,Crash Bandicoot,PS,1996,Platform,Sony Computer Entertainment,3.23,2.35,0.94,0.30,6.82
...,...,...,...,...,...,...,...,...,...,...
15922,Rayman 10th Anniversary,GBA,2005,Platform,Ubisoft,0.01,0.00,0.00,0.00,0.02
16162,Malice,XB,2004,Platform,Evolved Games,0.01,0.00,0.00,0.00,0.01
16241,Kao the Kangaroo,GBA,2001,Platform,Titus,0.01,0.00,0.00,0.00,0.01


In [12]:
vgsales[(vgsales.Genre.str.contains("Action")) & (vgsales.Year > 1995) & (vgsales.Year < 2006)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
18,Grand Theft Auto: San Andreas,PS2,2004,Action,Take-Two Interactive,9.43,0.40,0.41,10.57,20.81
25,Grand Theft Auto: Vice City,PS2,2002,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
39,Grand Theft Auto III,PS2,2001,Action,Take-Two Interactive,6.99,4.51,0.30,1.30,13.10
91,Grand Theft Auto: Liberty City Stories,PSP,2005,Action,Take-Two Interactive,2.90,2.83,0.24,1.75,7.72
95,The Legend of Zelda: Ocarina of Time,N64,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
...,...,...,...,...,...,...,...,...,...,...
16442,Shin Sangoku Musou 4 Special,X360,2005,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01
16450,Legacy of Kain: Defiance,PC,2003,Action,Eidos Interactive,0.00,0.01,0.00,0.00,0.01
16552,E.T. The Extra-Terrestrial,GBA,2001,Action,NewKidCo,0.01,0.00,0.00,0.00,0.01


In [13]:
vgsales[(vgsales.Genre.str.contains("Sports")) & (vgsales.Year > 1995) & (vgsales.Year < 2006)]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
203,Tony Hawk's Pro Skater,PS,1999,Sports,Activision,3.42,1.38,0.02,0.20,5.02
212,Madden NFL 06,PS2,2005,Sports,Electronic Arts,3.98,0.26,0.01,0.66,4.91
226,Tony Hawk's Pro Skater 2,PS,2000,Sports,Activision,3.05,1.41,0.02,0.20,4.68
239,Madden NFL 2005,PS2,2004,Sports,Electronic Arts,4.18,0.26,0.01,0.08,4.53
249,Tony Hawk's Pro Skater 3,PS2,2001,Sports,Activision,2.66,1.29,0.01,0.46,4.41
...,...,...,...,...,...,...,...,...,...,...
16354,World Snooker Challenge 2005,PSP,2005,Sports,Sega,0.00,0.01,0.00,0.00,0.01
16429,ESPN NBA 2Night 2002,XB,2002,Sports,Konami Digital Entertainment,0.01,0.00,0.00,0.00,0.01
16448,Wade Hixton's Counter Punch,GBA,2004,Sports,"Destination Software, Inc",0.01,0.00,0.00,0.00,0.01


A quantidade de lançamentos de jogos de plataforma é muito inferior a desses outros dois gêneros. Foram lançados quase o dobro de jogos de ação, e mais que o dobro de jogos de esporte (mas que individualmente não venderam tão bem quanto os dos outros).

A partir do ano de 2005, o gênero de ação toma a liderança, depois do lançamento de diversos sucessos consecutivos da franquia GTA, e tal liderança continua até os dias de hoje. Apesar da base de dados não ter dados bons o suficiente para representar os anos além de 2015, a tendência é clara: a disparidade entre a primeira e segunda colocações vem aumentando cada vez mais, e o mercado de jogos da atualidade não sofreu nenhuma outra mudança brusca o suficiente para desestablizar tal situação. 

In [16]:
vgsales[(vgsales.Genre.str.contains("Action")) & (vgsales.Year > 1996)].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
17,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
18,Grand Theft Auto: San Andreas,PS2,2004,Action,Take-Two Interactive,9.43,0.40,0.41,10.57,20.81
24,Grand Theft Auto V,X360,2013,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38
25,Grand Theft Auto: Vice City,PS2,2002,Action,Take-Two Interactive,8.41,5.49,0.47,1.78,16.15
39,Grand Theft Auto III,PS2,2001,Action,Take-Two Interactive,6.99,4.51,0.30,1.30,13.10
